## Import 

In [1]:
import tensorflow as tf
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, Activation, Dropout, Flatten, Dense

import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image
import numpy as np

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Users\utilisateur\anaconda3\envs\morpion\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-58fa1f01493b>", line 1, in <module>
    import tensorflow as tf
  File "C:\Users\utilisateur\anaconda3\envs\morpion\lib\site-packages\tensorflow\__init__.py", line 54, in <module>
    from ._api.v2 import compat
  File "C:\Users\utilisateur\anaconda3\envs\morpion\lib\site-packages\tensorflow\_api\v2\compat\__init__.py", line 39, in <module>
    from . import v1
  File "C:\Users\utilisateur\anaconda3\envs\morpion\lib\site-packages\tensorflow\_api\v2\compat\v1\__init__.py", line 34, in <module>
    from . import compat
  File "C:\Users\utilisateur\anaconda3\envs\morpion\lib\site-packages\tensorflow\_api\v2\compat\v1\compat\__init__.py", line 39, 

TypeError: object of type 'NoneType' has no len()

## Telechargement du ZIP

In [ ]:
from modules.loader import Loader

loader = Loader(
    "https://stdatalake010.blob.core.windows.net/public/cifar-100.zip",
    '../datas/ZIP/',
    extraction_target='../datas/RAW/'
)
loader.ensure_data_loaded()

## Extraction d'un jeu de donnees

In [ ]:
from modules.splitting import Splitting

labels_array = ['baby', 'tank', 'beaver', 'bear', 'bed', 'apple', 'aquarium_fish']

TRAIN_DATA_DIR = Splitting.copie_dossiers(
    '../datas/RAW/train',
    labels_array,
    500,
    explorer=False
)

## Chargement des images

In [ ]:
image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

# Taille d'image
TRAIN_IMAGE_SIZE = 32
# NB d'images envoyées à la fois
TRAIN_BATCH_SIZE = 64

train_generator = image_data_generator.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(TRAIN_IMAGE_SIZE, TRAIN_IMAGE_SIZE),
    batch_size=TRAIN_BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    )
 
validation_generator = image_data_generator.flow_from_directory(
    TRAIN_DATA_DIR, # same directory as training data
    target_size=(TRAIN_IMAGE_SIZE, TRAIN_IMAGE_SIZE),
    batch_size=TRAIN_BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

## Creation du modele

- convolution
- dense
- pooling

In [ ]:
model = Sequential()

model.add(Conv2D(16, kernel_size=3, activation='elu', kernel_initializer='he_uniform', padding='same', input_shape=(taille_image.shape)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dense(32, activation='elu'))
model.add(Dropout(0.2))

model.add(Conv2D(32, kernel_size=3, activation='elu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dense(64, activation='elu'))
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=3, activation='elu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dense(128, activation='elu'))
model.add(Dropout(0.2))


model.add(Conv2D(128, kernel_size=5, activation='elu', kernel_initializer='he_uniform', padding='same'))
model.add(Dense(256, activation='elu'))

#Toujours à la fin
model.add(Flatten())
model.add(Dense(7, activation='softmax'))

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es_callback = EarlyStopping(monitor='val_loss', patience=3)
training = model.fit(train_generator, epochs=20, callbacks=[es_callback], validation_data=validation_generator, shuffle=False)


In [ ]:
plt.plot(training.history['accuracy'], color='red', label='Training accuracy')
plt.plot(training.history['val_accuracy'],  color='green', label='Validation accuracy')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()
plt.ylim((.5,1.01))

plt.show()

In [ ]:
plt.plot(training.history['loss'], color='red', label='Training loss')
plt.plot(training.history['val_loss'],  color='green', label='Validation loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()

plt.show()

## Sauvegarde du modele

In [ ]:
model_name = 'model.h5'

model.save(model_name)

model.summary()

## Affichage des couches du modele

In [ ]:
from modules.observation_modele import plot_layer

img = "../datas/RAW/train/apple/0020.png"
plt.imshow(plt.imread(image))

plot_layer(model, image, [0,3,7,10])

In [ ]:
from modules.observation_modele import plot_layer

img = "../datas/RAW/train/tank/0103.png"
plt.imshow(plt.imread(image))

plot_layer(model, image, [0,3,7,10])